# Accessing Db2 with Go

This lab uses the **go_ibm_db** cli driver, which is already installed in the lab environment. The driver can be used to develop applications that are running on Linux, Mac, or Windows environments. It provides an interface for GoLang to Db2 for z/OS, Db2 for LUW, and Db2 for i.  
  
If you want to use the driver in your own environment you can [download it from GitHub](https://github.com/ibmdb/go_ibm_db). 

In [49]:
%run refresh.ipynb

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Table of Contents

* [Overview of the Db2-Go lab environment](#overview)
* [Connect to database SAMPLE](#connect)
* [Retrieve a single row from a table](#select_single)
* [Execute a query and fetch the result set](#fetch)
* [Retrieve the column names from the result set](#fetch_columns)
* [Additional sample Programs](#solutions)
    * [count3.go](#count3.go)

<a id='introduction'></a>
## Overview of the Db2-Go lab environment

You use database **SAMPLE** for the Db2-Go tutorials. The database is already installed in the lab environment. You can check [the schema description of the database](https://www.ibm.com/support/knowledgecenter/en/SSEPGG_11.1.0/com.ibm.db2.luw.apdv.samptop.doc/doc/r0001094.html).
Below is a list of the tables that are used in the labs together with the corresponding number of rows and columns.

| Tables                 | Number of Rows          | Number of Columns  |
|:---------------------- |:------------------------|:-------------------|
|ACT|18|3|
|DEPARTMENT|14|5|
|EMPLOYEE  |42|14|
|EMPPROJACT|73|6|
|ORG|8|5|
|PROJECT|20|9|
|ROLES|11|2|
|SALES|41|4|
|STAFF|3|7|

### How to Copy Code and Examples
Throughout this lab there are code samples that need to be copied and modified in a text editor. Any commands that need to be executed from a command line are found in grey boxes (an example is found below) has been designed to be easily copied.

In [50]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=111 onmousedown="window.clipline(111)" onmouseup="window.reset(111)">
Sample commands are found in cells like this.
</pre>
</div>

The entire contents of the text in the cell will be automatically copied when you click on the cell. The color of the background will change color briefly to indicate that the copy has completed. To paste commands into a terminal window, use the key combination Shift-Control-v. 

Code blocks that are meant to be placed into an editor will have a `COPY` button found at the bottom of the text. You can manually select the text you want to copy, or use the `COPY` button to place the entire contents onto the clipboard. 

<!--sample.code-->
```go
If (code == sample) then
   place onto clipboard
else
   ignore
end if
```

In [51]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('sample.code')">Copy</button> 

## Exploring the Contents of the Database
You can explore the database using either the Db2 command line, Visual Studio Code, or the Db2 Jupyter notebook extensions. To explore the database from the Db2 command line perform the following steps.

1) Open a shell window (click the *terminal* icon in the task bar below the Linux desktop) 
   ![](./media/terminal-selected.png)

   Use the following technique to make it easier to switch between a Jupyter notebook and the command line. When you open up the terminal window, right click on the title bar and select `Always on Top`. This will keep the terminal window on top of your Jupyter notebook so you can easily move between the two environments.
   
   ![](./media/alwaysontop.png)
   
   Remember to use `Shift-Control-v` when copying text into a command window.


2) Switch to user **db2inst1** (password: **db2inst1**). You can use the **db2pot** user to issue Db2 commands, but you will need to connect as user **DB2INST1**. 

In [52]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=100 onmousedown="window.clipline(100)" onmouseup="window.reset(100)">
su - db2inst1
</pre>
</div>

3) Start Db2. The database may already be started, but in the event that you receive an error when connecting in the next step, issue the following command.  

In [53]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=200 onmousedown="window.clipline(200)" onmouseup="window.reset(200)">
db2start
</pre>
</div>

4) Connect to database SAMPLE:  

In [54]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=300 onmousedown="window.clipline(300)" onmouseup="window.reset(300)">
db2 connect to sample
</pre>
</div>

5) Show the tables in schema DB2INST1:  

In [55]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=400 onmousedown="window.clipline(400)" onmouseup="window.reset(400)">
db2 "select substr(tabname,1,15) from syscat.tables where tabschema='DB2INST1'"
</pre>
</div>

6) Show the table definition of one of the tables, for example, table ACT:  

In [56]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=500 onmousedown="window.clipline(500)" onmouseup="window.reset(500)">
db2 "describe table db2inst1.act"
</pre>
</div>

7) Retrieve the rows from one of the tables, for example, table ACT:  

In [57]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=600 onmousedown="window.clipline(600)" onmouseup="window.reset(600)">
db2 "select * from db2inst1.act"
</pre>
</div>

8) When you are done, enter the command `exit` to return back to the `db2pot` environment.

In [58]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=700 onmousedown="window.clipline(700)" onmouseup="window.reset(700)">
exit
</pre>
</div>

## View the Database Contents Using Db2 Jupyter Extensions
The following code will load the Db2 Magic commands required to run SQL against Db2 from within a Jupyter Notebook.

In [ ]:
%run db2.ipynb
%run connection.ipynb

### List the Tables in the DB2INST1 Schema

In [ ]:
%sql -grid select substr(tabname,1,15) from syscat.tables where tabschema='DB2INST1'

### Describe the ACT Table

In [ ]:
%sql describe table act

### View the Contents of the ACT Table

In [ ]:
%sql -grid select * from act

## GO Language Environment

The following variables should be set in your Linux environment:
```bash
DB2HOME="/home/db2pot/go/src/github.com/ibmdb/go_ibm_db/installer/clidriver
CGO_CFLAGS=-I$DB2HOME/include
CGO_LDFLAGS=-L$DB2HOME/lib
LD_LIBRARY_PATH=/home/db2pot/go/src/github.com/ibmdb/go_ibm_db/installer/clidriver/lib
```

For OSX you require the following environment setting.
```bash
DYLD_LIBRARY_PATH=$DYLD_LIBRARY_PATH:/home/db2pot/go/src/github.com/ibmdb/go_ibm_db/installer/clidriver/lib
```

Before you start, make sure that the environment is set up correctly:
- Open a Linux shell
   ![](./media/terminal-selected.png)


- If you are not already logged on as user `db2pot`, switch to user `db2pot` (password: `123qwe123`). 
<br><p>**Note:** Execute all Go program examples as user **db2pot**.    

In [59]:
%%html
<div style="margin-left: 45px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=1000 onmousedown="window.clipline(1000)" onmouseup="window.reset(1000)">
su - db2pot
</pre>
</div>

- To check the value of an environment variable, for example `DB2HOME`, run the following command:  

In [60]:
%%html
<div style="margin-left: 45px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=1100 onmousedown="window.clipline(1100)" onmouseup="window.reset(1100)">
export | grep DB2HOME
</pre>
</div>

- The environment variables are defined in file `.profile` in directory `/home/db2pot`

In [61]:
%%html
<div style="margin-left: 45px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=1200 onmousedown="window.clipline(1200)" onmouseup="window.reset(1200)">
more /home/db2pot/.profile
</pre>
</div>

Execute all program examples from the Linux shell. To execute a program, take the following steps:

- Create directory `Labs` in the home directory of user db2pot (**Note:** This may already be done for you):  

In [62]:
%%html
<div style="margin-left: 45px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=1300 onmousedown="window.clipline(1300)" onmouseup="window.reset(1300)">
cd /home/db2pot
mkdir Labs
</pre>
</div>

- Copy the program code from the Jupyter notebook into a package file. The first line of each program example shows the program name that should be used when you create the corresponding program file. For example, for the below listed `hello_world.go` program create the corresponding program file as follows:  

In [63]:
%%html
<div style="margin-left: 45px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=1400 onmousedown="window.clipline(1400)" onmouseup="window.reset(1400)">
cd /home/db2pot/Labs
leafpad hello_world.go &
</pre>
</div>

- The `leafpad` editor can be found in the task bar if you need to access it outside of a command line
   ![](./media/leafpad-selected.png)

- Copy and paste the program code to the leafpad editor

<!--hello_world.go-->
```go
// hello_world.go
package main
import "fmt"
func main() {
    fmt.Println("Hello world")
}
```

In [64]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('hello_world.go')">Copy</button> 

- Click the `Save` button and name it `hello_world.go`


- Execute the program as follows:

In [65]:
%%html
<div style="margin-left: 45px; border-style: solid; border-width: 1px; background-color:#F2F2F2; padding: 10px;" >
<pre id=1600 onmousedown="window.clipline(1600)" onmouseup="window.reset(1600)">
go run hello_world.go
</pre>
</div>

<a id='connect'></a>
## Connect to database SAMPLE

You start with a simple program that connects to the `SAMPLE` database. It imports the following packages which are required to deploy the Db2 driver API:
```go
import _ "github.com/ibmdb/go_ibm_db"  
import "database/sql"  
```
**Note:** The underscore before the package `github.com/ibmdb/go_ibm_db` is required. It ensures that the `init` function of the package is executed and package-level variables are created.

The function `sql.Open()` is executed to setup a database connection. It requires the driver name `go_ibm_db` and the connection string `con` as input parameters. The connection string specifies hostname, port number, database name, user name, and password. 
If `sql.Open()` was executed successfully, the database handle `db` is initialized. Otherwise it will be set to `nil`. Before the program terminates it calls function `db.Close()`. It closes the database connection and cleans up the database handle.  
  
Execute the `connect.go` program example from the shell as described in the previous section.

<!--connect.go-->
```go
// connect.go

package main

import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)

func main(){
        con := "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"
        db, err:=sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
        }
        fmt.Println("Success!")
        db.Close()
}
```

In [66]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('connect.go')">Copy</button> 

Now adjust the code to make it more handy to use. Since the database handle `db` is always required to execute a Db2 API function, define the connection related variables `db`, `err`, and `con` outside of function `main()`. This makes sure you can access the database handle in all functions that are defined in package `main`.  

Also put the `db.Open()` call into a separate function `connect()`. 

Finally, make sure that function `db.Close()` is automatically called when function `main()` terminates. Put the `defer` keyword in front of the statement. This makes sure that `db.Close()` is automatically executed as soon as a return statement is executed anywhere in function `main()`.

The following program code contains all modifications that are described above. Execute `connect2.go` from the shell.

<!--connect2.go-->
```go
// connect2.go

package main

import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)

var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}

func main() {
        if connect() != nil { return } else { defer db.Close() }
    
        fmt.Println("Success!")
}
```

In [67]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('connect2.go')">Copy</button> 

<a id='select_single'></a>
## Retrieve a single row from a table

In the next example, you use function `db.QueryRow()` to run a query that is expected to return at most one row. If there multiple rows in the query result set,  the function will only access the first row and discard the rest.  
  
You use the following select statement in this example:  
  
```sql
select count(*) statement from act
```

This statement always returns exactly one row and returns the number of records in table ACT.

The `Scan()` function copies the columns from the current row into the values pointed. Since we expect a single integer value in the query result set, we define variable `count` of type `int32` and pass a pointer to that variable into function `scan()`.  
  
Execute program `count_in_main.go` from the shell.

<!--count_in_main.go-->
```go
// count_in_main.go

package main

import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)
var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}
func main() {
        if connect() != nil { return } else { defer db.Close() }

        var count int32
        err = db.QueryRow("SELECT count(*) FROM act").Scan(&count)
        if err != nil {
                fmt.Println(err)
                return
        }
        fmt.Println("Rowcount=",count)
}
```

In [68]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('count_in_main.go')">Copy</button> 

In the next example, the call to function `db.QueryRow()` is moved into a separate function `count()`. This function expects a tablename as a parameter. It prints the row count of the table and returns parameter `err`. 
This allows us to check in `main()` if an error occurred in function `count()`.

<!--count2.go-->
```go
// count2.go

package main
import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)
var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}
func count(tabname string) error { 
        var count int32
        err = db.QueryRow("SELECT count(*) FROM "+tabname).Scan(&count)
        if err != nil {
                fmt.Println(err)
                return err
        }       
        fmt.Printf("Table \"%s\" contains %d rows.\n",tabname,count)
        return nil
}

func main() {
        if connect() != nil { return } else { defer db.Close() }

        count("ACT")
        count("DEPARTMENT")
        count("EMPLOYEE")
        count("null")
        count("ORG")
}
```

In [69]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('count2.go')">Copy</button> 

You see that one of the table names in the list is invalid ("`null`"). In this case the program just prints an error message and continues to process. You could also do additional error handling in function `main()`. For example, you could remove the `Printf()` statement in function `count()` and then invoke `count()` in function `main()` as follows:  
```go
if count("ACT") != nil { fmt.Printf("Error in function count()") }
```
Try to rewrite program `count2.go`. Create a new program `count3.go` that performs the following steps:
- Stores the tablenames in a dynamic array
- Iterates over the array
- Passes each table name to function `count()`
- Checks the return value of function `count()` and does the error handling in function `main()`.

You will find the corresponding program `count3.go` at the end of this notebook.

<a id='fetch'></a>
## Execute a query and fetch the result set

Next, you learn how to run a `SELECT` statement that returns more than one row. In this example, we use the following select statement:  
  
 ```sql
 select userid,role from roles
 ```  
  
Function `db.Query()` prepares and executes the `SELECT` statement. Function `rows.Next()` iterates over the result set and prepares the next result row for reading with the `Scan()` api. Since the `SELECT` statement returns two values in each row of the result set, we define variables `a` and `b` and pass their address to function `Scan()`. The function copies the columns from the current row into variables `a` and `b`.

Execute program `fetch_rows.go` from the shell. The program will abort with an error. Try to modify the program code and fix the error.

<!--fetch_rows.go-->
```go
// fetch_rows.go

package main
import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)
var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}

func main() {
        if connect() != nil { return } else { defer db.Close() }

        rows,err := db.Query("select userid,role from roles")
        if err != nil {
                return
        }
        // make sure that the "rows" handle is released when main returns
        defer rows.Close()

        // iterate over all rows in the query result
        var a,b int32
        for rows.Next() {
                err = rows.Scan(&a,&b)
                if err != nil{
                        fmt.Println(err)
                        return
                }
                fmt.Printf("%s  %s\n",a,b)
        }
}
```

In [70]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('fetch_rows.go')">Copy</button> 

The program aborts with the following error message:  
  
```go
sql: Scan error on column index 0, name "USERID": converting driver.Value type []uint8 ("JAMES") to a int32: invalid syntax
```
  
You can fix the problem by changing the type of variables `a` and `b`. They are declared as `int32`. Since the `SELECT` statement returns string values the variables should be of type `string`.

<a id='fetch_columns'></a>
## Retrieve the column names from the result set

Db2 provides the option to rename table columns. So the names of columns may change over time. Therefore, it can be useful to retrieve the current column names from a table. 

To retrieve the column names, we use an SQL query that selects all columns from the table and retrieves one row only:  
  
 ```sql
 select * from employee fetch first 1 row only
 ```  
   
   
We use function `db.Query()` to execute that query. The function returns a handle to the query result (`rows`):

 ```go
 rows,err := db.Query("select * from employee fetch first 1 row only")
 ```  
  
We then use handle `rows` to retrieve the column names. Function `rows.Columns()` stores the column names in a dynamically created array:  
  
 ```go
 cols, err := rows.Columns()
 ```
  
We can use function `Printf()` to print the whole array at once:  
  
 ```go
 fmt.Printf("%v\n",cols) 
 ```
  
Alternatively, we can use the `range` operator to iterate over the array and print each element on a separate line. There are different ways to use the range operator:  
 ```go  
 for _,name := range cols { ... }
 for idx,name := range cols { ... }
 ```  

In our example, we use the first form which only retrieves the elements of the array `cols`. Alternatively, you can also retrieve the index value of each array element. Execute program `get_columns.go` from the shell. 

<!--get_columns.go-->
```go
// get_columns.go

package main
import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)

var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}
func main() {
        if connect() != nil { return } else { defer db.Close() }

        rows,err := db.Query("select * from employee fetch first 1 row only")
        if err != nil {
                fmt.Printf("db.Query(): error!")
                return
        }
        // make sure that the "rows" handle is released when main returns
        defer rows.Close()

        cols, err := rows.Columns()
        fmt.Println("Number of columns: ",len(cols))
        // print the whole array at once
        fmt.Printf("%v\n",cols)
        // print each column name on a separate line
        for _,name := range cols {
                fmt.Printf("%s\n",name)
        }
}
```

In [71]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('get_columns.go')">Copy</button> 

<a id='solutions'></a>
## Additional sample programs

<a id='count3.go'></a>
### count3.go

<!--count3.go-->
```go
// count3.go

package main
import (
    _ "github.com/ibmdb/go_ibm_db"
    "database/sql"
    "fmt"
)
var err error
var db *sql.DB
var con = "HOSTNAME=localhost;PORT=50000;DATABASE=SAMPLE;UID=DB2INST1;PWD=db2inst1"

func connect() error {
        db, err = sql.Open("go_ibm_db", con)
        if err != nil {
                fmt.Println(err)
                return err
        }
        return nil
}

func count(tabname string) error {
        var count int32
        err = db.QueryRow("SELECT count(*) FROM "+tabname).Scan(&count)
        if err != nil {
                return err
        }
        fmt.Printf("Table \"%s\" contains %d rows.\n",tabname,count)
        return nil
}

func main() {
        if connect() != nil { return } else { defer db.Close() }

        tabs := []string{"ACT","EMPLOYEE","null","ROLES","SALES","ORG","STAFF"}
        for index, tab := range tabs {
                fmt.Printf("%d\n", index)
                err = count(tab)
                if err != nil { fmt.Println(err) }
        }
}
```

In [72]:
%%html
<div style="margin-left: 45px">
<button type="button" onclick="window.copySearch('count3.go')">Copy</button> 

#### Credits: IBM 2019, Andreas Christian [achristian@de.ibm.com]